## Disney Call Option - valuation, delta, vega
QuantLib Closed Form

In [1]:
import QuantLib as ql

# option data
expiry_date = ql.Date(21, 7, 2023)
spot_price = 99.64
strike_price = 45
volatility = 1.3 # yahoo finance 5y monthly
dividend_rate =  0
option_type = ql.Option.Call

risk_free_rate = 0.0343 #10-year yield
day_count = ql.Actual365Fixed()
calendar = ql.UnitedStates(ql.UnitedStates.NYSE)

calculation_date = ql.Date(12, 4, 2023)
ql.Settings.instance().evaluationDate = calculation_date

In [2]:
# construct the European Option
payoff = ql.PlainVanillaPayoff(option_type, strike_price)
exercise = ql.EuropeanExercise(expiry_date)
european_option = ql.VanillaOption(payoff, exercise)

spot_handle = ql.QuoteHandle(
    ql.SimpleQuote(spot_price)
)
flat_ts = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, risk_free_rate, day_count)
)
dividend_yield = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, dividend_rate, day_count)
)
flat_vol_ts = ql.BlackVolTermStructureHandle(
    ql.BlackConstantVol(calculation_date, calendar, volatility, day_count)
)
bsm_process = ql.BlackScholesMertonProcess(spot_handle,
                                           dividend_yield,
                                           flat_ts,
                                           flat_vol_ts)

european_option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))

In [3]:
# calculate and display results
print("option value is", "%.3f" % european_option.NPV())
print("option delta is", "%.3f" % european_option.delta())
print("option vega is", "%.3f" % european_option.vega())

option value is 57.600
option delta is 0.936
option vega is 6.532


In [4]:
import yahoo_fin.options as ops
ops.get_calls("dis", "07/21/2023").iloc[0]

Contract Name             DIS230721C00045000
Last Trade Date       2023-04-14 10:14AM EDT
Strike                                  45.0
Last Price                              55.9
Bid                                     55.5
Ask                                    56.35
Change                                   0.0
% Change                                   -
Volume                                     1
Open Interest                             46
Implied Volatility                    93.65%
Name: 0, dtype: object

In [5]:
# The result is quite similar